In [4]:
import pandas as pd
import dask.dataframe as dd

In [5]:
d = [
    {
        'id':[2],
        'object_names':['bab'],
        'paths':['/bab'],
        'size':[12],
        'type':['zip'],
        'upload':[True]
    },
    {
        'id':[1],
        'object_names':['bob'],
        'paths':['/bob'],
        'size':[2],
        'type':['zip'],
        'upload':[False]
    },
    {
        'id':[0],
        'object_names':['bib'],
        'paths':['/bib'],
        'size':[9],
        'type':['file'],
        'upload':[True]
        }
]

In [15]:
ddf = dd.from_pandas(pd.DataFrame([], columns=['id', 'object_names', 'paths', 'size', 'type', 'upload']))

In [16]:
for de in d:
    df = pd.DataFrame(de)
    if (len(ddf.index) == 0):
        ddf = dd.from_pandas(df, npartitions=1)
    else:
        ddf = dd.concat([ddf, dd.from_pandas(df, npartitions=1)])

In [17]:
ddf

,id,object_names,paths,size,type,upload
npartitions=3,,,,,,
,int64,string,string,int64,string,bool
,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [19]:
del d[2]

In [20]:
d

[{'id': [2],
  'object_names': ['bab'],
  'paths': ['/bab'],
  'size': [12],
  'type': ['zip'],
  'upload': [True]},
 {'id': [1],
  'object_names': ['bob'],
  'paths': ['/bob'],
  'size': [2],
  'type': ['zip'],
  'upload': [False]}]

In [21]:
ddf.compute()

,id,object_names,paths,size,type,upload
0,2,bab,/bab,12,zip,True
0,1,bob,/bob,2,zip,False
0,0,bib,/bib,9,file,True


In [27]:
import os

In [29]:
path = os.path.abspath('../../../test_tree_large')

In [30]:
path

'/home/dave/test_tree_large'

In [80]:
df = pd.DataFrame()
folder_count = 0
file_count = 0
for folder, _, files in os.walk(path, topdown=True):
    if len(files) > 0:
        folder_count += 1
        file_count += len(files)
        # print(f'folder: {folder}')
        # print(f'sub_folders: {sub_folders}')
        # print(f'files: {files}')
        folder_files = [os.path.join(folder, file) for file in files]
        df = pd.concat([df, pd.DataFrame(folder_files, columns=['local_path'])], ignore_index=True)

In [81]:
df['size'] = df['local_path'].apply(lambda x: os.path.getsize(x))
df['object_names'] = df['local_path'].apply(lambda x: os.path.basename(x))
df['upload'] = False
df['type'] = 'file'

In [82]:
ddf = dd.from_pandas(df, npartitions=100)
ddf

,local_path,size,object_names,upload,type
npartitions=100,,,,,
0,string,int64,string,bool,string
90,...,...,...,...,...
...,...,...,...,...,...
8910,...,...,...,...,...
8999,...,...,...,...,...


In [83]:
current_objects = pd.Series(os.listdir('../../../test_tree_large/7/'))

In [84]:
current_objects = pd.DataFrame(current_objects, columns=['object_names'])
current_objects

,object_names
0,7_256.f
1,7_175.f
2,7_609.f
3,7_769.f
4,7_422.f
...,...
996,7_834.f
997,7_845.f
998,7_195.f
999,7_507.f


In [85]:
ddf['exists'] = ddf['object_names'].isin(current_objects['object_names'])

In [88]:
done = ddf.compute()

In [92]:
uploads = done[done['exists'] == False].drop(columns=['exists'])

In [93]:
uploads

,local_path,size,object_names,upload,type
0,/home/dave/test_tree_large/3/3_425.f,1048576,3_425.f,False,file
1,/home/dave/test_tree_large/3/3_805.f,1048576,3_805.f,False,file
2,/home/dave/test_tree_large/3/3_795.f,1048576,3_795.f,False,file
3,/home/dave/test_tree_large/3/3_238.f,1048576,3_238.f,False,file
4,/home/dave/test_tree_large/3/3_562.f,1048576,3_562.f,False,file
...,...,...,...,...,...
8995,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,False,file
8996,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,False,file
8997,/home/dave/test_tree_large/1/1_671.f,1048576,1_671.f,False,file
8998,/home/dave/test_tree_large/1/1_613.f,1048576,1_613.f,False,file


In [94]:
max_zip_size = 1048576 * 100

In [107]:
uploads

,local_path,size,object_names,upload,type,cumsize
0,/home/dave/test_tree_large/3/3_425.f,1048576,3_425.f,False,file,True
1,/home/dave/test_tree_large/3/3_805.f,1048576,3_805.f,False,file,True
2,/home/dave/test_tree_large/3/3_795.f,1048576,3_795.f,False,file,True
3,/home/dave/test_tree_large/3/3_238.f,1048576,3_238.f,False,file,True
4,/home/dave/test_tree_large/3/3_562.f,1048576,3_562.f,False,file,True
...,...,...,...,...,...,...
8995,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,False,file,NaN
8996,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,False,file,NaN
8997,/home/dave/test_tree_large/1/1_671.f,1048576,1_671.f,False,file,NaN
8998,/home/dave/test_tree_large/1/1_613.f,1048576,1_613.f,False,file,NaN


In [133]:

cumulative_size = 0
batches = []
batch = []
batch_number = []
for row in uploads.iterrows():
    size = row[1]['size']
    if cumulative_size + size > max_zip_size:
        batches.append(None)
        batch = [None]
        cumulative_size = size
    else:
        batch.append(None)
        cumulative_size += size
    if row[1].name == uploads.index[-1]:
        batches.append(None)
    batch_number.append(len(batches))
del batch, batches, cumulative_size
uploads['batch'] = batch_number

AttributeError: 'DataFrameGroupBy' object has no attribute 'capitalize'